In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import math
import copy
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('train.csv')
data.sample(5)

message_id  num_links  num_words  has_offer  sender_score  all_caps  \
5399         5400          0        101          0      0.784384         0   
11346       11347          1        164          0      0.406144         0   
2675         2676          1        184          0      0.844819         0   
14413       14414          2         97          0      0.683464         0   
1869         1870          4        110          0      0.627517         0   

       is_spam  
5399         0  
11346        0  
2675         0  
14413        0  
1869         0

**No need for the message_id col, won't help in classification**

In [4]:
data = data.drop('message_id', axis=1)

In [5]:
data.head()

num_links  num_words  has_offer  sender_score  all_caps  is_spam
0          3         98          1      0.718607         0        0
1          0        170          0      0.698901         1        0
2          0         38          0      0.620466         0        0
3          0        116          0      0.701755         0        0
4          3         89          1      0.583621         1        1

In [6]:
print(f"Missing values: {data.isna().sum()}")

Missing values: num_links       0
num_words       0
has_offer       0
sender_score    0
all_caps        0
is_spam         0
dtype: int64


In [7]:
print(f"Datatypes: {data.dtypes}")

Datatypes: num_links         int64
num_words         int64
has_offer         int64
sender_score    float64
all_caps          int64
is_spam           int64
dtype: object



In [8]:
X = data[['num_links', 'num_words', 'has_offer', 'sender_score', 'all_caps']]
y = data['is_spam']

In [9]:
from sklearn.preprocessing import StandardScaler

numerical_features = ['num_links', 'num_words', 'sender_score']
scaler = StandardScaler()
X[numerical_features] = scaler.fit_transform(X[numerical_features])

print(f"Scaled features sample: {X.head()}")

Scaled features sample:    num_links  num_words  has_offer  sender_score  all_caps
0   1.229832  -0.224189          1      0.129767         0
1  -1.227003   1.161143          0      0.025105         1
2  -1.227003  -1.378632          0     -0.391486         0
3  -1.227003   0.122144          0      0.040259         0
4   1.229832  -0.397355          1     -0.587176         1


In [10]:
# Quick class distribution check (no plot)
spam_count = np.sum(data['is_spam'] == 1)
non_spam_count = np.sum(data['is_spam'] == 0)
print(f"Class distribution: Non-Spam={non_spam_count}, Spam={spam_count}")
print(f"Imbalance ratio: {non_spam_count/spam_count:.1f}:1")

Class distribution: Non-Spam=17354, Spam=1746
Imbalance ratio: 9.9:1

Imbalance ratio: 9.9:1


**We can notice class imbalance!!**

In [11]:
#Weight balancing using inverse frequency
class_counts = np.bincount(y) #[17354, 1746]
print(class_counts,"\n")
print(len(y))
class_weights = len(y) / (class_counts * len(class_counts))

print(f"not-spam / spam: {class_weights}")

[17354  1746] 

19100
not-spam / spam: [0.55030541 5.4696449 ]


In [12]:
X_train, X_val, y_train, y_val = train_test_split(
    X.values, y, test_size=0.2, random_state=45, stratify=y
)

y_train = np.array(y_train)
y_val = np.array(y_val)

print("\nTraining set shape:", X_train.shape, y_train.shape)
print("Validation set shape:", X_val.shape, y_val.shape)
print("Training class distribution:", np.bincount(y_train) / len(y_train))
print("Validation class distribution:", np.bincount(y_val) / len(y_val))


Training set shape: (15280, 5) (15280,)
Validation set shape: (3820, 5) (3820,)
Training class distribution: [0.9085733 0.0914267]
Validation class distribution: [0.90863874 0.09136126]


**let's begin with our logistic regression**

In [24]:
# ENHANCED LOGISTIC REGRESSION WITH DYNAMIC CLASS WEIGHTS
def sigmoid(z):
    """Sigmoid activation function"""
    z = np.clip(z, -500, 500)  # Prevent overflow
    return 1 / (1 + np.exp(-z))

def compute_class_weights(y, weight_type='balanced'):
    """Compute dynamic class weights"""
    classes = np.unique(y)
    class_counts = np.bincount(y)
    
    if weight_type == 'balanced':
        # Standard balanced weights: n_samples / (n_classes * class_count)
        weights = len(y) / (len(classes) * class_counts)
    elif weight_type == 'inverse':
        # Simple inverse frequency
        weights = 1.0 / class_counts
        weights = weights / np.sum(weights) * len(classes)  # Normalize
    elif weight_type == 'log':
        # Log-scaled weights for extreme imbalance
        weights = np.log(len(y) / class_counts)
    else:  # uniform
        weights = np.ones(len(classes))
    
    # Create sample weights array
    sample_weights = np.zeros(len(y))
    for i, class_label in enumerate(classes):
        sample_weights[y == class_label] = weights[i]
    
    return sample_weights, weights

def compute_cost(X, y, w, b, lambda_l1=0.0, sample_weights=None):
    """Cost function with L1 regularization and class weights"""
    m = X.shape[0]
    z = np.dot(X, w) + b
    y_pred = sigmoid(z)
    
    # Prevent log(0) by clipping predictions
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    
    # Cross-entropy cost with sample weights
    if sample_weights is not None:
        cost = -np.mean(sample_weights * (y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred)))
    else:
        cost = -np.mean(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred))
    
    # L1 regularization (Lasso)
    l1_cost = lambda_l1 * np.sum(np.abs(w))
    
    return cost + l1_cost

def compute_gradients(X, y, w, b, lambda_l1=0.0, sample_weights=None):
    """Compute gradients for gradient descent with class weights"""
    m = X.shape[0]
    z = np.dot(X, w) + b
    y_pred = sigmoid(z)
    
    # Gradients with sample weights
    error = y_pred - y
    if sample_weights is not None:
        error = error * sample_weights
    
    dw = (1/m) * np.dot(X.T, error) + lambda_l1 * np.sign(w)
    db = (1/m) * np.sum(error)
    
    return dw, db

def gradient_descent(X, y, w, b, learning_rate=0.1, lambda_l1=0.0, num_iterations=1000, 
                    sample_weights=None):
    """Gradient descent optimization with class weights"""
    costs = []
    
    for i in range(num_iterations):
        # Compute cost and gradients
        cost = compute_cost(X, y, w, b, lambda_l1, sample_weights)
        dw, db = compute_gradients(X, y, w, b, lambda_l1, sample_weights)
        
        # Update parameters
        w = w - learning_rate * dw
        b = b - learning_rate * db
        
        # Store cost every 100 iterations
        if i % 100 == 0:
            costs.append(cost)
    
    return w, b, costs

def predict(X, w, b, threshold=0.5):
    """Make predictions"""
    z = np.dot(X, w) + b
    y_pred_proba = sigmoid(z)
    y_pred = (y_pred_proba >= threshold).astype(int)
    return y_pred, y_pred_proba

def evaluate_model(y_true, y_pred):
    """Calculate evaluation metrics"""
    tp = np.sum((y_true == 1) & (y_pred == 1))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1_score': f1,
        'tp': tp, 'fp': fp, 'fn': fn, 'tn': tn
    }

def log_loss(y_true, y_pred):
    eps  = 1e-15
    y_pred = np.clip(y_pred, eps,  1 - eps)
    return -np.mean(y_true*np.log(y_pred) + (1-y_true)*np.log(1 - y_pred))

In [ ]:
#evaluation metrics
def print_results(metrics, title="Model Results"):
    """Print evaluation metrics in a clean format"""
    print(f"\n{title}")
    print("="*50)
    print(f"Accuracy:  {metrics['accuracy']:.4f} ({metrics['accuracy']*100:.2f}%)")
    print(f"Precision: {metrics['precision']:.4f}")
    print(f"Recall:    {metrics['recall']:.4f}")
    print(f"F1 Score:  {metrics['f1_score']:.4f}")
    print(f"\nConfusion Matrix:")
    print(f"  TP: {metrics['tp']:4d} | FP: {metrics['fp']:4d}")
    print(f"  FN: {metrics['fn']:4d} | TN: {metrics['tn']:4d}")

In [16]:
# COMPREHENSIVE GRID SEARCH WITH DYNAMIC CLASS WEIGHTS
print("GRID SEARCH FOR OPTIMAL HYPERPARAMETERS")
print("="*50)

# Define parameter grids (4 values each)
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5, 1.0],
    'lambda_l1': [0.0, 0.001, 0.01, 0.1], 
    'num_iterations': [1000, 2000, 3000, 5000],
    'weight_type': ['uniform', 'balanced', 'inverse', 'log']
}

print(f"Grid Search Parameters:")
for param, values in param_grid.items():
    print(f"  {param}: {values}")

# Initialize tracking variables
best_f1 = 0
best_params = {}
results_log = []
total_combinations = 1
for values in param_grid.values():
    total_combinations *= len(values)

# Grid search with cross-validation on validation set
print("\nStarting Grid Search...")
combination_count = 0

for lr in param_grid['learning_rate']:
    for lambda_l1 in param_grid['lambda_l1']:
        for num_iter in param_grid['num_iterations']:
            for weight_type in param_grid['weight_type']:
                combination_count += 1
                
                try:
                    # Compute dynamic class weights
                    sample_weights, class_weights = compute_class_weights(y_train, weight_type)
                    
                    # Initialize weights
                    np.random.seed(42)
                    n_features = X_train.shape[1]
                    w = np.random.normal(0, 0.01, n_features)
                    b = 0.0
                    
                    # Train model
                    w_trained, b_trained, cost_history = gradient_descent(
                        X_train, y_train, w, b,
                        learning_rate=lr, lambda_l1=lambda_l1, 
                        num_iterations=num_iter, sample_weights=sample_weights
                    )
                    
                    # Evaluate on validation set (using default threshold 0.5 for fair comparison)
                    y_pred_val, y_pred_proba_val = predict(X_val, w_trained, b_trained, threshold=0.5)
                    metrics = evaluate_model(y_val, y_pred_val)
                    
                    # Log results
                    result = {
                        'learning_rate': lr,
                        'lambda_l1': lambda_l1, 
                        'num_iterations': num_iter,
                        'weight_type': weight_type,
                        'f1_score': metrics['f1_score'],
                        'accuracy': metrics['accuracy'],
                        'precision': metrics['precision'],
                        'recall': metrics['recall'],
                        'final_cost': cost_history[-1] if cost_history else float('inf')
                    }
                    results_log.append(result)
                    
                    # Track best model
                    if metrics['f1_score'] > best_f1:
                        best_f1 = metrics['f1_score']
                        best_params = {
                            'learning_rate': lr,
                            'lambda_l1': lambda_l1,
                            'num_iterations': num_iter,
                            'weight_type': weight_type,
                            'weights': w_trained,
                            'bias': b_trained,
                            'metrics': metrics
                        }
                    
                    # Simple progress output with error tracking
                    final_cost = cost_history[-1] if cost_history else float('inf')
                    print(f"LR: {lr} | L1: {lambda_l1} | Iter: {num_iter} | Weight: {weight_type} | Error: {final_cost:.4f} | F1: {metrics['f1_score']:.4f}")

                except Exception as e:
                    print(f"Error in combination {combination_count}: {str(e)}")
                    continue

print(f"Best F1 Score: {best_f1:.4f}")
print(f"Best Parameters:")
for param, value in best_params.items():
    if param not in ['weights', 'bias', 'metrics']:
        print(f"  {param}: {value}")

GRID SEARCH FOR OPTIMAL HYPERPARAMETERS
Grid Search Parameters:
  learning_rate: [0.01, 0.1, 0.5, 1.0]
  lambda_l1: [0.0, 0.001, 0.01, 0.1]
  num_iterations: [1000, 2000, 3000, 5000]
  weight_type: ['uniform', 'balanced', 'inverse', 'log']

Starting Grid Search...
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: uniform | Error: 0.3010 | F1: 0.0112
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: uniform | Error: 0.3010 | F1: 0.0112
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: balanced | Error: 0.4700 | F1: 0.3801
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: balanced | Error: 0.4700 | F1: 0.3801
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: inverse | Error: 0.1827 | F1: 0.3103
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: inverse | Error: 0.1827 | F1: 0.3103
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: log | Error: 0.1387 | F1: 0.2216
LR: 0.01 | L1: 0.0 | Iter: 1000 | Weight: log | Error: 0.1387 | F1: 0.2216
LR: 0.01 | L1: 0.0 | Iter: 2000 | Weight: uniform | Error: 0.2547 | F1: 0.0490
LR: 0.01 | L1: 0.0 | Iter: 200

In [17]:
# FINAL MODEL TRAINING WITH BEST PARAMETERS
print("TRAINING FINAL MODEL WITH BEST PARAMETERS")
print("="*45)

# Use best parameters found from grid search
BEST_LR = best_params['learning_rate']
BEST_LAMBDA = best_params['lambda_l1'] 
BEST_ITER = best_params['num_iterations']
BEST_WEIGHT_TYPE = best_params['weight_type']

print(f"Final Model Parameters:")
print(f"  Learning Rate: {BEST_LR}")
print(f"  L1 Regularization: {BEST_LAMBDA}")
print(f"  Iterations: {BEST_ITER}")
print(f"  Weight Type: {BEST_WEIGHT_TYPE}")

# Final training with best parameters
sample_weights_final, class_weights_final = compute_class_weights(y_train, BEST_WEIGHT_TYPE)

print(f"\nFinal Class Weights ({BEST_WEIGHT_TYPE}):")
print(f"  Class 0 (Non-spam): {class_weights_final[0]:.3f}")
print(f"  Class 1 (Spam): {class_weights_final[1]:.3f}")

# Train final model
np.random.seed(42)
n_features = X_train.shape[1]
w_final = np.random.normal(0, 0.01, n_features)
b_final = 0.0

w_trained, b_trained, cost_history = gradient_descent(
    X_train, y_train, w_final, b_final,
    learning_rate=BEST_LR, lambda_l1=BEST_LAMBDA, 
    num_iterations=BEST_ITER, sample_weights=sample_weights_final
)

print(f"\nFinal Training Complete!")
print(f"Final Cost: {cost_history[-1]:.4f}")
print(f"Cost Reduction: {cost_history[0]:.4f} -> {cost_history[-1]:.4f}")
print(f"Convergence: {'Good' if cost_history[-1] < cost_history[0] * 0.5 else 'Check'}")

TRAINING FINAL MODEL WITH BEST PARAMETERS
Final Model Parameters:
  Learning Rate: 0.5
  L1 Regularization: 0.0
  Iterations: 5000
  Weight Type: uniform

Final Class Weights (uniform):
  Class 0 (Non-spam): 1.000
  Class 1 (Spam): 1.000

Final Training Complete!
Final Cost: 0.1473
Cost Reduction: 0.6935 -> 0.1473
Convergence: Good

Final Training Complete!
Final Cost: 0.1473
Cost Reduction: 0.6935 -> 0.1473
Convergence: Good


In [26]:
# MODEL EVALUATION ON VALIDATION SET
print("EVALUATING TRAINED MODEL ON VALIDATION DATA")
print("="*50)

# validation set predictions
y_pred_val, y_pred_proba_val = predict(X_val, w_trained, b_trained)

# Calculate evaluation metrics
val_metrics = evaluate_model(y_val, y_pred_val)

#check the log loss
val_log_loss = log_loss(y_val, y_pred_proba_val)
print(f"Log loss: {val_log_loss:.5f}")

# Display results using our helper function
print_results(val_metrics, "🏆 VALIDATION SET PERFORMANCE")



EVALUATING TRAINED MODEL ON VALIDATION DATA
Log loss: 0.16237

🏆 VALIDATION SET PERFORMANCE
Accuracy:  0.9374 (93.74%)
Precision: 0.7148
Recall:    0.5244
F1 Score:  0.6050

Confusion Matrix:
  TP:  183 | FP:   73
  FN:  166 | TN: 3398


In [19]:
# LOAD AND PREPROCESS TEST DATA
print("LOADING TEST DATA FOR PREDICTIONS")
print("="*40)

# Load test data
test_data = pd.read_csv('test.csv')
print(f"Test data shape: {test_data.shape}")
print(f"Test data columns: {list(test_data.columns)}")

# Extract message IDs for submission
test_message_ids = test_data['message_id'].values
print(f"Message ID range: {test_message_ids.min()} to {test_message_ids.max()}")

# Prepare features (same as training)
X_test = test_data[['num_links', 'num_words', 'has_offer', 'sender_score', 'all_caps']]

# Apply same scaling as training data
X_test_scaled = X_test.copy()
X_test_scaled[numerical_features] = scaler.transform(X_test[numerical_features])

print(f"Test features shape: {X_test_scaled.shape}")
print(f"Test features sample:")
print(X_test_scaled.head())

LOADING TEST DATA FOR PREDICTIONS
Test data shape: (900, 6)
Test data columns: ['message_id', 'num_links', 'num_words', 'has_offer', 'sender_score', 'all_caps']
Message ID range: 20000 to 20899
Test features shape: (900, 5)
Test features sample:
   num_links  num_words  has_offer  sender_score  all_caps
0  -0.408058  -1.397873          0     -0.319948         0
1  -1.227003   1.238106          0     -0.618016         0
2  -1.227003   0.122144          0     -1.583159         0
3  -1.227003   0.045181          0     -0.253475         0
4   0.410887  -0.339633          1      1.344932         1
Test data shape: (900, 6)
Test data columns: ['message_id', 'num_links', 'num_words', 'has_offer', 'sender_score', 'all_caps']
Message ID range: 20000 to 20899
Test features shape: (900, 5)
Test features sample:
   num_links  num_words  has_offer  sender_score  all_caps
0  -0.408058  -1.397873          0     -0.319948         0
1  -1.227003   1.238106          0     -0.618016         0
2  -1.22700

In [20]:
# MAKE PREDICTIONS ON TEST DATA
print("GENERATING PREDICTIONS ON TEST DATA")
print("="*40)

# Use trained model to make predictions
y_test_pred, y_test_proba = predict(X_test_scaled.values, w_trained, b_trained)

print(f"Predictions generated for {len(y_test_pred)} samples")
print(f"Predicted spam count: {np.sum(y_test_pred)}")
print(f"Predicted non-spam count: {np.sum(1 - y_test_pred)}")
print(f"Spam percentage: {np.mean(y_test_pred)*100:.1f}%")

# Show some sample predictions
print(f"\nSample predictions:")
for i in range(5):
    spam_status = "SPAM" if y_test_pred[i] == 1 else "NOT SPAM"
    confidence = y_test_proba[i] if y_test_pred[i] == 1 else (1 - y_test_proba[i])
    print(f"  Message {test_message_ids[i]}: {spam_status} (confidence: {confidence:.3f})")

GENERATING PREDICTIONS ON TEST DATA
Predictions generated for 900 samples
Predicted spam count: 63
Predicted non-spam count: 837
Spam percentage: 7.0%

Sample predictions:
  Message 20000: NOT SPAM (confidence: 0.999)
  Message 20001: NOT SPAM (confidence: 1.000)
  Message 20002: NOT SPAM (confidence: 0.999)
  Message 20003: NOT SPAM (confidence: 1.000)
  Message 20004: SPAM (confidence: 0.751)


In [21]:
print("CREATING SUBMISSION CSV FILE WITH SPAM PROBABILITIES")
print("="*55)


submission_df = pd.DataFrame({
    'message_id': test_message_ids,
    'is_spam': y_test_proba  
})

submission_df['is_spam'] = submission_df['is_spam'].round(4)

print(f"Submission file shape: {submission_df.shape}")
print(f"Columns: {list(submission_df.columns)}")
print(f"Probability range: {submission_df['is_spam'].min():.4f} to {submission_df['is_spam'].max():.4f}")

print(f"\nFirst 5 rows (probabilities):")
print(submission_df.head())

print(f"\nLast 5 rows (probabilities):")
print(submission_df.tail())

print(f"\nProbability distribution:")
print(f"  Mean probability: {submission_df['is_spam'].mean():.4f}")
print(f"  Median probability: {submission_df['is_spam'].median():.4f}")
print(f"  High confidence spam (>0.8): {np.sum(submission_df['is_spam'] > 0.8)}")
print(f"  Low confidence non-spam (<0.2): {np.sum(submission_df['is_spam'] < 0.2)}")

# Save to CSV file
submission_filename = 'submission.csv'
submission_df.to_csv(submission_filename, index=False)

print(f"\nSubmission file saved as '{submission_filename}' with probabilities!")
print(f"File contains {len(submission_df)} probability predictions")
print(f"Ready for submission!")

CREATING SUBMISSION CSV FILE WITH SPAM PROBABILITIES
Submission file shape: (900, 2)
Columns: ['message_id', 'is_spam']
Probability range: 0.0001 to 0.9996

First 5 rows (probabilities):
   message_id  is_spam
0       20000   0.0010
1       20001   0.0003
2       20002   0.0006
3       20003   0.0003
4       20004   0.7509

Last 5 rows (probabilities):
     message_id  is_spam
895       20895   0.1816
896       20896   0.0013
897       20897   0.0582
898       20898   0.1101
899       20899   0.0015

Probability distribution:
  Mean probability: 0.0952
  Median probability: 0.0049
  High confidence spam (>0.8): 20
  Low confidence non-spam (<0.2): 760

Submission file saved as 'submission.csv' with probabilities!
File contains 900 probability predictions
Ready for submission!

Submission file saved as 'submission.csv' with probabilities!
File contains 900 probability predictions
Ready for submission!
